In [1]:
import gurobipy as gp
from gurobipy import GRB

# Data
cycle_times = [6.51,5.46,5.98,8.13,4.96,5.12,8.71,4.17,3.85,8.46,7.03,4.01,5.08,5.37,6.05,4.6,9.22,5.58,7.43,3.53,5.09,4,3.61,3.05,3.41
               ]
operator_counts = [38,37,36,35,34]
shift_hours = 9
n = len(cycle_times)

for total_ops in operator_counts:
    print(f'\n=== Total Operators: {total_ops} ===\n')
    
    # 1) Integer allocation
    m_int = gp.Model(f'Int_Alloc_{total_ops}')
    m_int.Params.OutputFlag = 0
    ops_int = m_int.addVars(n, vtype=GRB.INTEGER, lb=1, name='ops_int')
    mu_int = m_int.addVar(vtype=GRB.CONTINUOUS, name='mu_int')
    for i, T in enumerate(cycle_times):
        m_int.addConstr(60 * ops_int[i] >= mu_int * T)
    m_int.addConstr(ops_int.sum() == total_ops)
    m_int.setObjective(mu_int, GRB.MAXIMIZE)
    m_int.optimize()
    thr_int = mu_int.X
    daily_int = thr_int * shift_hours
    adj_daily_int = 0.72 * daily_int
    rates_int = [60 * ops_int[i].X / cycle_times[i] for i in range(n)]
    print('Integer Allocation:')
    print(f'▶ Throughput: {thr_int:.2f} u/hr, Output: {daily_int:.1f} units')
    print(f'▶ Adjusted for 28% PF&D: {adj_daily_int:.1f} units')
    print('| Station |   Ops |  Units/hr |')
    for i in range(n):
        print(f'|   {i+1:>2}    |  {int(ops_int[i].X):>3}  |   {rates_int[i]:>6.2f}  |')
    top3_int = sorted(range(n), key=lambda i: rates_int[i])[:3]
    print('Top 3 bottlenecks:')
    for idx in top3_int:
        print(f'  Station {idx+1}: {rates_int[idx]:.2f} u/hr')
    
    print('\n')
    
    # 2) Half-allocation
    m_half = gp.Model(f'Half_Alloc_{total_ops}')
    m_half.Params.OutputFlag = 0
    h_ops = m_half.addVars(n, vtype=GRB.INTEGER, lb=1, name='h_ops')
    mu_half = m_half.addVar(vtype=GRB.CONTINUOUS, name='mu_half')
    for i, T in enumerate(cycle_times):
        m_half.addConstr(60 * h_ops[i] >= mu_half * (2 * T))
    m_half.addConstr(h_ops.sum() == total_ops * 2)
    m_half.setObjective(mu_half, GRB.MAXIMIZE)
    m_half.optimize()
    thr_half = mu_half.X
    daily_half = thr_half * shift_hours
    adj_daily_half = 0.72 * daily_half
    rates_half = [(60 * (h_ops[i].X / 2)) / cycle_times[i] for i in range(n)]
    print('Half Allocation:')
    print(f'▶ Throughput: {thr_half:.2f} u/hr, Output: {daily_half:.1f} units')
    print(f'▶ Adjusted for 28% PF&D: {adj_daily_half:.1f} units')
    print('| Station |   Ops |  Units/hr |')
    for i in range(n):
        ops_val = h_ops[i].X / 2
        print(f'|   {i+1:>2}    | {ops_val:>5.2f} |   {rates_half[i]:>6.2f}  |')
    top3_half = sorted(range(n), key=lambda i: rates_half[i])[:3]
    print('Top 3 bottlenecks:')
    for idx in top3_half:
        print(f'  Station {idx+1}: {rates_half[idx]:.2f} u/hr')
    
    print('\n')
    
    # 3) Fractional allocation
    m_frac = gp.Model(f'Frac_Alloc_{total_ops}')
    m_frac.Params.OutputFlag = 0
    ops_frac = m_frac.addVars(n, vtype=GRB.CONTINUOUS, lb=0, name='ops_frac')
    mu_frac = m_frac.addVar(vtype=GRB.CONTINUOUS, name='mu_frac')
    for i, T in enumerate(cycle_times):
        m_frac.addConstr(60 * ops_frac[i] >= mu_frac * T)
    m_frac.addConstr(ops_frac.sum() == total_ops)
    m_frac.setObjective(mu_frac, GRB.MAXIMIZE)
    m_frac.optimize()
    thr_frac = mu_frac.X
    daily_frac = thr_frac * shift_hours
    adj_daily_frac = 0.72 * daily_frac
    rates_frac = [60 * ops_frac[i].X / cycle_times[i] for i in range(n)]
    print('Fractional Allocation:')
    print(f'▶ Throughput: {thr_frac:.2f} u/hr, Output: {daily_frac:.1f} units')
    print(f'▶ Adjusted for 28% PF&D: {adj_daily_frac:.1f} units')
    print('| Station |   Ops |  Units/hr |')
    for i in range(n):
        print(f'|   {i+1:>2}    | {ops_frac[i].X:>5.2f} |   {rates_frac[i]:>6.2f}  |')
    
    print('\n' + '-'*40 + '\n')


=== Total Operators: 38 ===

Set parameter Username
Set parameter LicenseID to value 2663177
Academic license - for non-commercial use only - expires 2026-05-09
Integer Allocation:
▶ Throughput: 11.79 u/hr, Output: 106.1 units
▶ Adjusted for 28% PF&D: 76.4 units
| Station |   Ops |  Units/hr |
|    1    |    2  |    18.43  |
|    2    |    2  |    21.98  |
|    3    |    2  |    20.07  |
|    4    |    2  |    14.76  |
|    5    |    1  |    12.10  |
|    6    |    2  |    23.44  |
|    7    |    2  |    13.78  |
|    8    |    1  |    14.39  |
|    9    |    1  |    15.58  |
|   10    |    2  |    14.18  |
|   11    |    2  |    17.07  |
|   12    |    1  |    14.96  |
|   13    |    1  |    11.81  |
|   14    |    2  |    22.35  |
|   15    |    2  |    19.83  |
|   16    |    1  |    13.04  |
|   17    |    2  |    13.02  |
|   18    |    2  |    21.51  |
|   19    |    2  |    16.15  |
|   20    |    1  |    17.00  |
|   21    |    1  |    11.79  |
|   22    |    1  |    15.00  |
